<div style="font-size: 200%; font-weight: bold; color: maroon;">Linear Regression with python + numpy. Boston housing example</div>


Shamelessly adapted from (with great thanks, of course):

https://www.cs.toronto.edu/~lczhang/321/tut/tut02_ta.html

We will:

    set up the linear regression problem using numpy
    show that vectorized code is faster 
    solve the linear regression problem using the closed form solution
    solve the linear regression problem using gradient descent 


In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

### Importing dataset from scikit-learn

In [ ]:
from sklearn.datasets import load_boston
housing_data = load_boston()

**The dataset is a dictionary and now let's see its keys.**

In [ ]:
print(housing_data.keys())

In [ ]:
# Let's check description of the dataset
print(housing_data["DESCR"])

**To keep the example simple, we will only work with two features: INDUS and RM. The explanations of these features are in the description above.**

**Notation for our matrices**

X = predictors, features, generally a matrix

y = target, generally a vector

w = weights, a vector of coefficients (w1 ... wp)

b = or beta0, intercept or constant in the linear model. Also bias -requires explanation.

yhat = prediction of the target when we have some w and b estimates

e = yhat - y = error or residual of the model (at some moment)

(caveat this notation is different from the used in the original notebook)


In [ ]:
# take the boston data
data = housing_data['data']
# we will only work with two of the features: INDUS and RM
X = data[:, [2,5]]
y = housing_data['target']

Just to give us an intuition of how these two features INDUS and RM affect housing prices, lets visualize the feature interactions. As expected, the more "industrial" a neighbourhood is, the lower the housing prices. The more rooms houses in a neighbourhood have, the higher the median housing price.

In [ ]:
# Individual plots for the two features:
plt.title('Industrialness vs Med House Price')
plt.scatter(X[:, 0], y)
plt.xlabel('Industrialness')
plt.ylabel('Med House Price')
plt.show()

plt.title('Avg Num Rooms vs Med House Price')
plt.scatter(X[:, 1], y)
plt.xlabel('Avg Num Rooms')
plt.ylabel('Med House Price')
plt.show()



## Defining the Cost Function

In lecture, we defined the cost function for a linear regression problem using the square loss:

$$\mathcal{E}(y, yhat) = \frac{1}{2N} \sum_{i=1}^N (y^{(i)} - yhat^{(i)})^2$$

In our case, since we have two features $x_1$ and $x_2$, our linear regression model will look like this:

$$\mathcal{E}(y, yhat) = \frac{1}{2N} \sum_{i=1}^N (y^{(i)} - (w_1 x_1^{(i)} + w_2 x_2^{(i)} + b) )^2$$

Note that b denotes the intercept. If you want to use the "beta0" notation this is the same:

$$\mathcal{E}(y, yhat) = \frac{1}{2N} \sum_{i=1}^N (y^{(i)} - (w_1 x_1^{(i)} + w_2 x_2^{(i)} + beta0) )^2$$


We can use the above formula to compute the cost function across an entire dataset (X, y):

In [ ]:
def cost(w1, w2, b, X, y):
    '''
    Evaluate the cost function in a non-vectorized manner for 
    inputs `X` and targets `y`, at weights `w1`, `w2` and `b`.
    '''

    costs = 0
    for i in range(len(y)):
        yhat = w1 * X[i, 0] + w2 * X[i, 1] + b
        y_i = y[i]
        costs += 0.5 * (y_i - yhat) ** 2
    return costs / len(y)

For example, the cost for this hypothesis

**THIS IS AN EXAMPLE OF A UNIT TEST**

In [ ]:
cost(3, 5, 20, X, y)

...is higher than this one:

In [ ]:
cost(3, 5, 0., X, y)

## Vectorizing the cost function:

Vectorization is a way to use linear algebra to represent computations like the one above.
In Python, vectorized code written in numpy tend to be faster than code that uses a `for` loop.


If we write the linear regression cost function using matrix computations, it would look like this:

$$\mathcal{E}(y, yhat) = \frac{1}{2N} \| y - (\bf{X} \bf{w} + b \bf{1}) \| ^2$$ 

**NOTE we are computing the result of multiplying b (a constant) by a vector of ones (1)**

Following the above formula, our vectorized code looks like this:

In [ ]:
def cost_vectorized(w1, w2, b, X, yhat):
    '''
    Evaluate the cost function in a vectorized manner for 
    inputs `X` and targets `t`, at weights `w1`, `w2` and `b`.
    '''

    N = len(y)
    w = np.array([w1, w2])
    yhat = np.dot(X, w) + b * np.ones(N)
    return np.sum((y - yhat)**2) / (2.0 * N)

Note we are using the b (intercept, bias, constant) outside the w vector. 

But we can also include it in the w vector (as it happens more often):

$$\mathcal{E}(y, yhat) = \frac{1}{2N} \| y - \bf{X} \bf{w}  \| ^2$$ 


However note the consequence of having only 2 columns in X. We must add a vector of 1's so that we can (matrix) multiply X by w (dimensions -or shape- of X (n, p+1), of w (1, p+1))

In [ ]:
def cost_vectorized_2(w1, w2, b, X, yhat):
    '''
    Evaluate the cost function in a vectorized manner for 
    inputs `X` and targets `t`, at weights `w1`, `w2` and `b`.
    '''

    N = len(y)
    w = np.array([b, w1, w2])
#    yhat = np.dot(X, w)
    X_1 = np.concatenate([np.ones([N, 1]),
                          X], 
                       axis=1)
    yhat = np.dot(X_1, w)
    return np.sum((y - yhat)**2) / (2.0 * N)

We can check that the vectorized code provides the same answers as the non-vectorized code:

In [ ]:
cost_vectorized(3, 5, 20, X, y)

In [ ]:
cost_vectorized(3, 5, 0, X, y)

In [ ]:
cost_vectorized_2(3, 5, 20, X, y)

In [ ]:
cost_vectorized_2(3, 5, 20, X, y)

In [ ]:
cost_vectorized(3, 5, 0, X, y)

## Comparing speed of the vectorized vs unvectorized code

We'll see below that the vectorized code already
runs ~2x faster than the non-vectorized code! 

Hopefully this will convince you to always vectorized your code whenever possible

In [ ]:
import time

Time for non-vectorized code:

In [ ]:
t0 = time.time()
print(cost(4, 5, 20, X, y))
t1 = time.time()
t2 = t1 - t0
print(t2)

Time for vectorized code:

In [ ]:
t0 = time.time()
print(cost_vectorized(4, 5, 20, X, y))
t1 = time.time()
t3 = t1 - t0
print(t3)

In [ ]:
print(t3/t2)

## Plotting cost in weight space

We'll plot the cost for two of our weights, assuming that bias = -22.89831573.

We'll see where that number comes from later.

Notice the shape of the contours are ovals.

In [ ]:
w1s = np.arange(-1.0, 0.0, 0.01)
w2s = np.arange(6.0, 10.0, 0.1)
z_cost = []
for w2 in w2s:
    z_cost.append([cost_vectorized(w1, w2, -22.89831573, X, y) for w1 in w1s])
z_cost = np.array(z_cost)
np.shape(z_cost)
W1, W2 = np.meshgrid(w1s, w2s)
CS = plt.contour(W1, W2, z_cost, 25)
plt.clabel(CS, inline=1, fontsize=10)
plt.title('Costs for various values of w1 and w2 for b=0')
plt.xlabel("w1")
plt.ylabel("w2")
plt.plot([-0.33471389], [7.82205511], 'o') # this will be the minima that we'll find later
plt.show()

## Exact Solution

Linear regression model has an exact solution, given by (all matrix formula)

$$\mathcal w = (X^{T} X)^{-1} X y $$ 

To better see this in the numpy code below, we will call:

$$\mathcal A = (X^{T} X) $$ 

$$\mathcal c = X y $$ 


**But first remember we must add a column of 1's to the features matrix to be able to compute the beta0 or intercept**

In [ ]:
# add an extra feature (column in the input) that are just all ones
X_1 = np.concatenate([np.ones([np.shape(X)[0], 1]),
                      X], 
                      axis=1)
X_1

In [ ]:
X_1.shape

In [ ]:
def solve_exactly(X, yhat):
    '''
    Solve linear regression exactly. (fully vectorized)
    
    Given `X` - NxD matrix of inputs
          `t` - target outputs
    Returns the optimal weights as a D-dimensional vector
    '''

    N, D = np.shape(X)
    A = np.matmul(X.T, X)
    c = np.dot(X.T, yhat)
    return np.matmul(np.linalg.inv(A), c)

In [ ]:
solve_exactly(X_1, y)

In practice, we use library code that is written for us.

In [ ]:
# In real life we don't want to code it directly
np.linalg.lstsq(X_1, y)

## Gradient Function and Gradient Descent

Another approach to optimize the cost function is via gradient descent.

The main idea is that we compute the gradient of the cost function with respect to
each parameter $w_j$, which will tell us how to update $w_j$ (by a small amount) to
improve the cost function (by a small amount).

In order to implement gradient descent, we need to be able to compute the *gradient*
of the cost function with respect to a weight $w_j$:

$$\frac{\partial \mathcal{E}}{\partial w_j} = \frac{1}{N}\sum_i x_j^{(i)}(yhat^{(i)}-y^{(i)})$$

which in matrix notation will be

$$\frac{\partial \mathcal{E}}{\partial w} = \frac{1}{N} (X^{T}(yhat - y))$$


In [ ]:
# Vectorized gradient function
def gradfn(weights, X_1, y):
    '''
    Given `weights` - a current "Guess" of what our weights should be
          `X_1` - matrix of shape (N,D) of input features with an additional column of 1s
          `y` - target y values
    Return gradient of each weight evaluated at the current value
    '''

    N, D = np.shape(X_1)
    yhat = np.matmul(X_1, weights)
    error = yhat - y
    return np.matmul(np.transpose(X_1), error) / float(N)

With this function, we can solve the optimization problem by repeatedly
applying gradient descent on $w$:

In [ ]:
def solve_via_gradient_descent(X_1, 
                               y, 
                               print_every = 5000,
                               niter = 100000, 
                               alpha = 0.005):
    '''
    Given ``X_1` - matrix of shape (N,D) of input features with an additional column of 1s
          `y` - target y values
    Solves for linear regression weights.
    Return weights after `niter` iterations.
    With an additional parameter alpha or learning_rate
    '''

    N, D = np.shape(X_1)
    # initialize all the weights to zeros
    w = np.zeros([D])
    for k in range(niter):
        dw = gradfn(w, X_1, y)
        w = w - alpha*dw
        if k % print_every == 0:
            print('Weight after %d iteration: %s' % (k, str(w)))
    return w

In [ ]:
solve_via_gradient_descent( X_1, y)

For comparison, this was the exact result:

In [ ]:
np.linalg.lstsq(X_1, y)